In [ ]:
%%timeit

import pandas as pd
import numpy as np
from typing import List, Dict, Union, Tuple
from functools import reduce

from pandas import DataFrame as DF
#from altair import Chart # for type aliasing

gh_raw_prefix = 'https://raw.githubusercontent.com/quinn-dougherty/well/master/'

csv_local = ['train_features.csv', 'test_features.csv', 'train_labels.csv', 'sample_submission.csv']
csv_github = {x: gh_raw_prefix + x for x in csv_local}

df = pd.read_csv(csv_github[csv_local[0]]).sample(8000)
y = pd.read_csv(csv_github[csv_local[2]])

to_drop = ['scheme_name', 'wpt_name', 'name']

null_vals_list = ['Not Known', 'Unknown', 'None', 'Not known', 'not known',
                  '-', 'unknown', 'Unknown Installer', '##', 'none', '0']

boolski = ['public_meeting', 'permit']

to_bin = {'construction_year': [-1, 1980, 1990, 2000, 2010, 2020], 
          'population': [-1, 10, 20, 100, 250, 1000, 5000, 10000, 100000]}

insigs = ['funder', 'installer', 'subvillage', 'ward']

dates = ['date_recorded']

twoprodc = ['region', 'district_code']
t = (twoprodc[0], twoprodc[1])

def const_impute(datf: DF, nl: List[str], const: str = "NOT_KNOWN") -> DF: 
  '''This is a naive and destructive fillna impute. 
  Given a dataframe and a list of strings known to be synonymous with "null"
  mutate the dataframe by replacing both missing vals and occurrences of vals in the given list 
  with a constant such as "NOT_KNOWN"
  
  also return the dataframe. '''
  return datf.replace({x: np.nan for x in nl}).fillna(const)

def bool_to_numeric(datf: DF, feats: List[str], unknown: float = 0.5) -> DF:
  """Bool to numeric"""
  mapped = datf[feats].replace({True: 1, False: 0, 'NOT_KNOWN': unknown})
  return datf.assign(**{name: mapped[name] for name in feats})

def binnit(datf: DF, 
           feats: Dict[str, List[int]], 
           ordinal: bool = True, 
           todrop: List[str] = to_drop) -> DF: 
  ''' binning'''
  print("doing binning")
  assert all([len(x)<26 for x in feats.values()])
  
  k: List[str] = list(feats.keys())

  todrop += k
  
  j: int = 2 * min([len(x) for x in k]) // 3
  
  if ordinal: 
    labels: Dict[str, List[int]] = {name: list(range(1,len(feats[name]))) for name in k}
  else: 
    labels: Dict[str, List[int]] = {name: [ch for ch 
                                           in "abcdefghijklmnopqrstuvwxyz"]
                                    [:len(feats[name])-1] 
              for name in k}
  
  for name in k: 
    a: str = name[:j]+'_binned'
    feats[a] = feats[name]
    
  return datf.assign(**{name[:j]+'_binned': pd.cut(datf[name],
                                                 bins=feats[name],#break
                                                 labels=labels[name]) 
                        for name in k})


def date_to_ord(datf: DF, feats: Union[str, List[str]]) -> DF: 
  print("doing dates")
  if isinstance(feats, str): 
    feats = [feats]

  return datf.assign(**{name: lambda df: (pd
                                          .to_datetime(df[name])
                                          .apply(lambda x: x.toordinal())) 
                        for name in feats})

def product(datf: DF, feats: Tuple[str,str], todrop: List[str] = to_drop) -> DF: 
  ''' df['region_district'] = df.apply(lambda row: f'{row.region}_{row.district_code}', axis=1) '''
  m = min([len(x) for x in feats])
  
  todrop += list(feats)
  
  name = f'{feats[0][:m]}_{feats[1][:m]}'
  
  return datf.assign(name = lambda df: [f'{le}_{ri}' 
                                        for le,ri in zip(df[feats[0]], df[feats[1]])])
  
# df['region_district'] = df.apply(lambda row: f'{row.region}_{row.district_code}', axis=1)
  pass


def insignificant(datf: DF, 
                  insgs: List[str], 
                  thresh: int = 3, 
                  fillconst: str = "OTHER") -> DF: 
  ''' WARNING: this code doesnt work without impute coming before it! '''
  print("doing insig drop")
  return datf.assign(**{name: [val 
                               if (datf[name].str.lower()
                                   .value_counts()[val]) > thresh
                               else fillconst
                               for val 
                               in datf[name].str.lower()] 
                        for name in insgs})


def droppem(datf: DF, droppin: List[str]) -> DF:
  #geog_outliers = [abs(x)+abs(y)>np.exp(-6) 
  #                 for x,y in zip(datf.longitude, datf.latitude)]
  return datf.drop(droppin, axis=1)#[geog_outliers]


X = droppem(product(insignificant(date_to_ord(binnit(bool_to_numeric(const_impute(df, 
                                                                                  null_vals_list), 
                                                                     boolski), 
                                                     to_bin), 
                                              dates), 
                                  insigs, thresh=2), 
                    t), 
            to_drop).replace({"NOT_KNOWN": np.nan})

print(to_drop)
print(X.shape)

X.head()

doing binning
doing dates
doing insig drop


In [ ]:

#X.to_csv("data", index=False)
y = pd.read_csv(csv_github[csv_local[2]])


#X.join(y, on='id')
X.merge(y.replace({"functional": 1, "non functional": -1, "functional needs repair": 0}), on='id').to_csv('data-large.csv', index=False)

In [19]:
y.shape

(59400, 2)